In [6]:
import numpy as np
from scipy.integrate import odeint

class Par:
    F = 100  # Example value for Par.F, adjust as needed

def train_model1(t, X):
    global Par
    
    # Model of The Real System
    x = X[:10]
    A = np.array([
        [0,    0,     0,    0,    0,     1,      0,      0,    0,    0],
        [0,    0,     0,    0,    0,     1,     -1,      0,    0,    0],
        [0,    0,     0,    0,    0,     0,      1,     -1,    0,    0],
        [0,    0,     0,    0,    0,     0,      0,      1,   -1,    0],
        [0,    0,     0,    0,    0,     0,      0,      0,    1,   -1],
        [0, -12.5,    0,    0,    0,  -0.75,   0.75,     0,    0,    0],
        [0,  62.5, -62.5,   0,    0,   3.75,  -7.5,   3.75,    0,    0],
        [0,     0,  62.5, -62.5,  0,     0,   3.75,  -7.5,  3.75,    0],
        [0,     0,     0,  62.5, -62.5,   0,     0,   3.75, -7.5,  3.75],
        [0,     0,     0,    0,  62.5,   0,     0,     0,   3.75, -3.75]
    ])
    
    b1 = np.array([0, 0, 0, 0, 0, 0.005, 0, 0, 0, 0])  # Force input
    b2 = np.array([0, 0, 0, 0, 0, 250, 0, 0, 0, -1250])  # constant input
    
    if t < 10:
        u = Par.F  # Constant Force
        uh = 0.5 * u
    else:
        u = 0
        uh = u
    
    xp = np.dot(A, x) + b1 * u + b2
    
    # Measurement matrix C and measurement dy
    C = np.array([
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    ])
    y = np.dot(C, x)
    dy = np.array([y[0] - 20, y[1]])
    
    # Model of The observer
    xh = X[10:]
    Ah = np.array([
        [0,    0,     0,    0,    0,     1,     -1,      0,      0,    0],
        [0,    0,     0,    0,    0,     0,      1,     -1,      0,    0],
        [0,    0,     0,    0,    0,     0,      0,      1,     -1,    0],
        [0,    0,     0,    0,    0,     0,      0,      0,      1,   -1],
        [0, -12.5,    0,    0,    0,  -0.75,   0.75,     0,      0,    0],
        [0,  62.5, -62.5,   0,    0,   3.75,  -7.5,   3.75,     0,    0],
        [0,     0,  62.5, -62.5,  0,     0,   3.75,  -7.5,   3.75,    0],
        [0,     0,     0,  62.5, -62.5,   0,     0,   3.75,  -7.5,   3.75],
        [0,     0,     0,    0,  62.5,   0,     0,     0,   3.75, -3.75]
    ])
    
    Bh = np.array([0, 0, 0, 0, 0.005, 0, 0, 0, 0])
    Ch = np.array([
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    ])
    
    yh = np.dot(Ch, xh)
    
    G = np.array([
        [10.5008, 0.0472],
        [4.0624, 0.0100],
        [1.2245, 0.0004],
        [0.3222, -0.0007],
        [118.1098, 1.1441],
        [60.1867, 0.5240],
        [16.7939, 0.3003],
        [-0.0227, 0.2370],
        [-4.2587, 0.2213]
    ])
    
    xhp = np.dot(Ah, xh) + np.dot(Bh, uh) + np.dot(G, (dy - yh))
    
    # Augment the real and estimated states
    Xp = np.concatenate((xp, xhp))
    
    return Xp



# Define initial conditions and time span
x0 = np.zeros(19)  # Initial state vector, adjust as needed
t_span = np.linspace(0, 20, 1000)  # Time span for simulation

# Integrate the system of differential equations
X = odeint(train_model1, x0, t_span)

# Extract states for plotting or further analysis
x_real = X[:, :10]  # Real states
x_estimated = X[:, 10:]  # Estimated states

TypeError: 'float' object is not subscriptable